In [48]:
import pandas as pd
import numpy as np
from skimage import io
from tqdm.notebook import tqdm
import os
import pickle
from sklearn.cluster import DBSCAN
from torchvision.models import efficientnet_v2_m
import torch.nn as nn
import torch
from torchvision import transforms

In [49]:
# Connect to the GPU if one exists.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using: ", device)
torch.cuda.empty_cache()

Using:  cuda


In [50]:
def get_seal_sub_images(img_path, water_classifier, model, step, device=device):
    # crops like reading a book
    centroids = []

    img = io.imread(img_path, plugin="matplotlib")
    
    x_len = img.shape[1]
    y_len = img.shape[0]

    model.to(device)
    transform = transforms.Compose([transforms.ToTensor()])

    # Iterate through y axis (Up to Down)
    i = 0
    while (i < y_len):
        # If y coordinate is greater than len of image, look at the last step of image (y-coords = (ymax, ymax-step))
        y1 = i
        if i + step > y_len:
            y1 = y_len-step
            y2 = y_len
            i = y_len
        # Increase y coords by step
        else:
            y2 = i + step
            i += step

        # Iterate through x axis (left to right)
        j = 0
        while (j < x_len):
            # If x coordinate is greater than len of image, look at the last step of image (x-coords = (xmax, xmax-step))
            x1 = j
            if j + step > x_len:
                x1 = x_len - step
                x2 = x_len
                j = x_len
            # Increase x coords by step
            else:
                x2 = j + step
                j += step

            # Sub images for models
            sub_image = np.array(
                    img[y1:y2, x1:x2]
                )
            sub_image_water_classifier = np.array(
                    [
                    np.array([sub_image]) # Wrapping in another array
                    .reshape(22500, 3)
                    .mean(axis=0)
                    ]
                )
            
            # Model predictions
            water_prediction = water_classifier.predict(sub_image_water_classifier)
            if water_prediction < .5:
                # Get subimage for CNN
                sub_image_cnn = (
                    transform(sub_image)
                    .unsqueeze(0)
                    .to(device)
                    )
                
                # Get seal prediction
                seal_prediction = np.argmax(
                    model(sub_image_cnn)
                        .cpu()
                        .detach()
                        .numpy(), 
                    axis = 1
                )[0]

                if seal_prediction > .5:

                    x_center = (x1 + x2) / 2
                    y_center = (y1 + y2) / 2
                    centroids.append((x_center, y_center))
                    
    return centroids

def get_file_names(path):
    names = set()
    for x in os.listdir(path):
        names.add(x.split(".")[0])
    return list(names)

def get_predicted_sub_images(cnn, water_classifier, path, write_path = None, version = ""):
    seal_centroids = {}

    file_names = get_file_names(path)

    # Get Centriods
    for file_name in tqdm(file_names, desc = "Determining Images with Seals"):
        img_name =  f"{file_name}.JPG"
        centroids = get_seal_sub_images(path+img_name, water_classifier, cnn, 150)
        seal_centroids[file_name] = centroids

    # Save file
    if write_path != None:
        with open(write_path + f"/seals_centroids_{version}.pkl", "wb") as f:
            pickle.dump(seal_centroids, f)
            
    return seal_centroids

In [51]:
# Load CNN
efficientnet = efficientnet_v2_m()
efficientnet.classifier[1] = nn.Linear(in_features=1280, out_features=2)
efficientnet.load_state_dict(torch.load(r"C:\Users\kaanan\Desktop\Noses\Pytorch Image Classifiers\ImageClassifierPytorch9"))
efficientnet.eval()
efficientnet = efficientnet.to(device)

In [52]:
f = open(r"C:\Users\kaanan\Desktop\Water Images and XML\water_classifier4", "rb")
water_classifier = pickle.load(f)
f.close()

In [53]:
# Load Paths to images
training_path = r"C:\Users\kaanan\Desktop\Training, Val, and Test Images\Training Images/"
validation_path = r"C:\Users\kaanan\Desktop\Training, Val, and Test Images\Validation Images/"
testing_path = r"C:\Users\kaanan\Desktop\Training, Val, and Test Images\Test Images/"

In [54]:
write_path = r"C:\Users\kaanan\Desktop\RCNN\MetaData" 


centroids_training = get_predicted_sub_images(efficientnet, water_classifier, training_path, write_path=write_path, version="training")
centroids_validation = get_predicted_sub_images(efficientnet, water_classifier, validation_path, write_path=write_path, version="validation")
centroids_testing= get_predicted_sub_images(efficientnet, water_classifier, testing_path, write_path=write_path, version="testing")

Determining Images with Seals:   0%|          | 0/50 [00:00<?, ?it/s]

Determining Images with Seals:   0%|          | 0/16 [00:00<?, ?it/s]

Determining Images with Seals:   0%|          | 0/13 [00:00<?, ?it/s]